### Data Processing
DataPreparing.ipynbで準備したデータ

s3://###default bucket###/xgboost-churn-stepfunctions/xgboost-churn/churn.txt

#### preprocessing

In [ ]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()

# データをS3から取得
import boto3
s3 = boto3.resource('s3')
s3.Bucket(bucket).download_file('xgboost-churn-stepfunctions/xgboost-churn/churn.txt', 'churn.txt')
sagemaker.__version__

上記セルを実行して、SageMaker Python SDK Version が 1.xx.x の場合、以下のセルのコメントアウトを解除してから実行してください。実行が完了したら、上にあるメニューから [Kernel] -> [Restart kernel] を選択してカーネルを再起動してください。

再起動が完了したら、このノートブックの一番上のセルから再度実行してください。その場合、以下のセルを実行する必要はありません。

In [ ]:
# !pip install -U --quiet "sagemaker==2.16.1"

#### ライブラリのセットアップ

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import re

In [ ]:
import pandas as pd
churn = pd.read_csv('./churn.txt')
pd.set_option('display.max_columns', 500)
churn


データをみると 3,333 行のデータしかなく、現在の機械学習の状況から見ると、やや小さいデータセットです。各データのレコードは、ある米国の携帯電話会社の顧客のプロフィールを説明する21の属性からなります。その属性というのは、

- State: 顧客が居住している米国州で、2文字の省略形で記載されます (OHとかNJのように)
- Account Length: アカウントが利用可能になってからの経過日数
- Area Code: 顧客の電話番号に対応する3桁のエリアコード
- Phone: 残りの7桁の電話番号
- Int’l Plan: 国際電話のプランに加入しているかどうか (yes/no)
- VMail Plan: Voice mail の機能を利用しているかどうか (yes/no)
- VMail Message: 1ヶ月の Voice mail のメッセージの平均長
- Day Mins: 1日に通話した時間(分)の総和
- Day Calls: 1日に通話した回数の総和
- Day Charge: 日中の通話にかかった料金
- Eve Mins, Eve Calls, Eve Charge: 夜間通話にかかった料金
- Night Mins, Night Calls, Night Charge: 深夜通話にかかった料金
- Intl Mins, Intl Calls, Intl Charge: 国際通話にかかった料金
- CustServ Calls: カスタマーサービスに電話をかけた回数
- Churn?: そのサービスから離反したかどうか (true/false)

最後の属性 Churn? は目的変数として知られ、MLのモデルで予測する属性になります。目的変数は2値 (binary) なので、ここで作成するモデルは2値の予測を行います。これは2値分類といわれます。

それではデータを詳しく見てみます。

まずはカテゴリデータごとにデータの頻度をみてみます。カテゴリデータは、State, Area code, Phone, Int’l Plan, VMail Plan, Churn?で、カテゴリを表す文字列や数値がデータとして与えられているものです。pandasではある程度自動で、カテゴリデータを認識し、objectというタイプでデータを保存します。以下では、object 形式のデータをとりだして、カテゴリごとの頻度を表示します。

また describe()を利用すると各属性の統計量を一度に見ることができます。

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

データを見てみると以下のことに気づくと思います。

- State の各頻度はだいたい一様に分布しています。
- Phone はすべて同じ数値になっていて手がかりになりそうにありません。この電話番号の最初の3桁はなにか意味がありそうですが、その割当に意味がないのであれば、使うのは止めるべきでしょう
- たった14%の顧客が離反しているので、インバランスなデータと言えるでしょうが、そこまで極端ではありません
- 数値的な特徴量は都合の良い形で分布しており、多くは釣り鐘のようなガウス分布をしています。ただ、VMail Messageは例外です。
- Area code は数値データとみなされているようなので、非数値に変換しましょう

さて、実際にPhoneの列を削除して、Area codeを非数値に変換します。

In [ ]:
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)

それでは次に各属性の値を、目的変数の True か False か、にわけて見てみます。

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column], columns=churn['Churn?'], normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

データ分析の結果から、離反する顧客について、以下のような傾向が考えられます。

- 地理的にもほぼ一様に分散している
- 国際通話を利用している
- VoiceMailを利用していない
- 通話時間で見ると長い通話時間と短い通話時間の人に分かれる
- カスタマーサービスへの通話が多い (多くの問題を経験した顧客ほど離反するというのは理解できる)

加えて、離反する顧客に関しては、Day Mins と Day Charge で似たような分布を示しています。しかし、話せば話すほど、通常課金されるので、驚くことではないです。もう少し深く調べてみましょう。corr() を利用すると相関係数を求めることができます。

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

いくつかの特徴は互いに100%の相関をもっています。このような特徴があるとき、機械学習のアルゴリズムによっては全くうまくいかないことがあり、そうでなくても結果が偏ったりしてしまうことがあります。これらの相関の強いペアは削除しましょう。Day Mins に対する Day Charge、Night Mins に対する Night Charge、Intl Mins に対する Intl Charge を削除します。

In [ ]:
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

ここまででデータセットの前処理は完了です。これから利用する機械学習のアルゴリズムを決めましょう。前述したように、数値の大小 (中間のような数値ではなく)で離反を予測するような変数を用意すると良さそうです。線形回帰のようなアルゴリズムでこれを行う場合は、複数の項（もしくはそれらをまとめた項）を属性として用意する必要があります。

そのかわりに、これを勾配ブースティング木 (Gradient Boosted Tree)を利用しましょう。Amazon SageMaker は、マネージドで、分散学習が設定済みで、リアルタイム推論のためのホスティングも可能な XGBoost コンテナを用意しています。XGBoost は、特徴感の非線形な関係を考慮した勾配ブースティング木を利用しており、特徴感の複雑な関連性を扱うことができます。

Amazon SageMaker の XGBoostは、csv または LibSVM 形式のデータを学習することができます。ここでは csv を利用します。csv は以下のようなデータである必要があります。

- 1列目が予測対象のデータ
- ヘッダ行はなし

まずはじめに、カテゴリ変数を数値データに変換する必要があります。get_dummies() を利用すると数値データへの変換が可能です。

そして、Churn?_Trueのデータを最初の列にもってきて、Churn?_False., Churn?_True.のデータを削除した残りのデータをconcatenate (連結) します。

In [ ]:
model_data = pd.get_dummies(churn)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)

ここで学習用、バリデーション用、テスト用データにわけましょう。これによって overfitting (学習用データには精度が良いが、実際に利用すると制度が悪い、といった状況) を回避しやすくなり、未知のテストデータに対する精度を確認することができます。

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
test_data.to_csv('test.csv', header=False, index=False)

In [ ]:
sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path='train.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn')
input_validation = sagemaker_session.upload_data(path='validation.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn')
input_test = sagemaker_session.upload_data(path='test.csv', key_prefix='xgboost-churn-stepfunctions/xgboost-churn')

In [ ]:
print(input_train)
print(input_validation)
print(input_test)

### 前処理をpythonスクリプトに書き出す

### SageMaker Processing

In [ ]:
import boto3
# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
print(region)
print(account_id)
ecr_repository = 'xgboost-churn-processing'
tag = ':latest'
nlpsample_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
# リポジトリの作成
# すでにある場合はこのコマンドは必要ありません。
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
!docker build -t xgboost-churn-processing .

In [ ]:
# docker imageをecrにpush
!docker tag {ecr_repository + tag} $nlpsample_repository_uri
!docker push $nlpsample_repository_uri

### localで実行の確認

In [ ]:
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
role = get_execution_role()

script_processor = ScriptProcessor(
    image_uri='%s.dkr.ecr.ap-northeast-1.amazonaws.com/%s:latest' % (account_id, ecr_repository),
    role=role,
    command=['python3'],
    instance_count=1,
    instance_type='ml.m5.xlarge')

In [ ]:
script_processor.run(code='processing.py',
    inputs=[ProcessingInput(
        source='churn.txt',
        destination='/opt/ml/processing/input')],
        outputs=[
        ProcessingOutput(source='/opt/ml/processing/output/data')]
)